In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython import display
%matplotlib inline

dfBostonCalendar = pd.read_csv('./Boston/calendar.csv')
dfBostonListing = pd.read_csv('./Boston/listings.csv')
dfBostonReview = pd.read_csv('./Boston/reviews.csv')
dfSeattleCalendar = pd.read_csv('./Seattle/calendar.csv')
dfSeattleListing = pd.read_csv('./Seattle/listings.csv')
dfSeattleReview = pd.read_csv('./Seattle/reviews.csv')

In [2]:
dfBostonCalendar.head()

,listing_id,date,available,price
0,12147973,2017-09-05,f,NaN
1,12147973,2017-09-04,f,NaN
2,12147973,2017-09-03,f,NaN
3,12147973,2017-09-02,f,NaN
4,12147973,2017-09-01,f,NaN


In [3]:
dfBostonListing['bedrooms'].value_counts()

1.0    2379
2.0     693
0.0     287
3.0     155
4.0      45
5.0      16
Name: bedrooms, dtype: int64

In [4]:
dfBostonListing.loc[dfBostonListing.id == 5506, ['bedrooms']]

,bedrooms
780,1.0


In [5]:
dfBostonListing['id'].is_unique

True

In [6]:
#convert the Boston Listing with available as 0 and nonavilable as 1 to check how many day has it been rented out
dfBostonCalendar.loc[dfBostonCalendar.available == 't', 'Rented Out'] = 0
dfBostonCalendar.loc[dfBostonCalendar.available == 'f', 'Rented Out'] = 1

In [7]:
#Create new table that display the average day of the each listing that has been rented out
dfBostonPriceEffectAnalysis = pd.DataFrame({'Percentage Rented Out': dfBostonCalendar.groupby(['listing_id']).mean()['Rented Out']}).reset_index()

In [8]:
dfBostonPriceEffectAnalysis.head()

,listing_id,Percentage Rented Out
0,3353,0.317808
1,5506,0.057534
2,6695,0.112329
3,6976,0.126027
4,8792,0.320548


In [9]:
#left join to get the number of room to the dfBostonPriceEffectAnalysis table
dfBostonPriceEffectAnalysis.set_index('listing_id')
dfBostonPriceEffectAnalysis2 = dfBostonPriceEffectAnalysis.join(other=dfBostonListing[['id','bedrooms', 'price']].set_index('id'), how='left', on='listing_id')

In [10]:
dfBostonPriceEffectAnalysis2.head()

,listing_id,Percentage Rented Out,bedrooms,price
0,3353,0.317808,1.0,$40.00
1,5506,0.057534,1.0,$145.00
2,6695,0.112329,1.0,$195.00
3,6976,0.126027,1.0,$65.00
4,8792,0.320548,1.0,$154.00


In [11]:
#conver price to float value
dfBostonPriceEffectAnalysis2['price'] = dfBostonPriceEffectAnalysis2['price'].apply(lambda x: x.replace('$','')).apply(lambda x: x.replace(',','')).astype(float)

In [12]:
dfBostonPriceEffectAnalysis2.describe()

,listing_id,Percentage Rented Out,bedrooms,price
count,3.585000e+03,3585.000000,3575.000000,3585.000000
mean,8.440875e+06,0.508640,1.255944,173.925802
std,4.500787e+06,0.389414,0.753060,148.331321
min,3.353000e+03,0.000000,0.000000,10.000000
25%,4.679319e+06,0.109589,1.000000,85.000000
50%,8.577620e+06,0.509589,1.000000,150.000000
75%,1.278953e+07,0.947945,2.000000,220.000000
max,1.493346e+07,1.000000,5.000000,4000.000000


In [13]:
#drop the row that has no value of bedroom
dfBostonPriceEffectAnalysis2.dropna(how='any', inplace=True)

In [14]:
dfBostonPriceEffectAnalysis2.describe()

,listing_id,Percentage Rented Out,bedrooms,price
count,3.575000e+03,3575.000000,3575.000000,3575.000000
mean,8.462666e+06,0.509770,1.255944,173.807552
std,4.488059e+06,0.389325,0.753060,148.511574
min,3.353000e+03,0.000000,0.000000,10.000000
25%,4.714580e+06,0.109589,1.000000,85.000000
50%,8.595650e+06,0.509589,1.000000,150.000000
75%,1.279715e+07,0.947945,2.000000,220.000000
max,1.493346e+07,1.000000,5.000000,4000.000000


In [15]:
#catgorize the price Below 150 is low price, between 150 5o 300 is mid price, above 300 is high price
dfBostonPriceEffectAnalysis2.loc[dfBostonPriceEffectAnalysis2.price <= 150, 'Price Cateogry'] = "Low Price"
dfBostonPriceEffectAnalysis2.loc[((dfBostonPriceEffectAnalysis2.price > 150) & (dfBostonPriceEffectAnalysis2.price <= 300)), 'Price Cateogry'] = "Medium Price"
dfBostonPriceEffectAnalysis2.loc[dfBostonPriceEffectAnalysis2.price > 300, 'Price Cateogry'] = "High Price"

In [16]:
#Drop the listing id column
dfBostonPriceEffectAnalysis2.drop(['listing_id'], axis=1, inplace=True)

In [17]:
#drop the price column
dfBostonPriceEffectAnalysis2.drop(['price'], axis=1, inplace=True)

In [18]:
#Display the result
dfBostonPriceEffectAnalysis2.groupby(['Price Cateogry', 'bedrooms']).mean().unstack()

Percentage Rented Out                                          \
bedrooms                         0.0       1.0       2.0       3.0       4.0   
Price Cateogry                                                                 
High Price                  0.333333  0.526568  0.385287  0.406897  0.288280   
Low Price                   0.479952  0.517093  0.568516  0.462709  0.635616   
Medium Price                0.523397  0.518135  0.549769  0.435582  0.702740   

                          
bedrooms             5.0  
Price Cateogry            
High Price      0.374315  
Low Price       1.000000  
Medium Price    0.576517

In [19]:
#Answer the question for the optimized room for AirBnB
dfBostonPriceEffectAnalysis2.groupby(['bedrooms', 'Price Cateogry']).mean().unstack()

Percentage Rented Out                       
Price Cateogry            High Price Low Price Medium Price
bedrooms                                                   
0.0                         0.333333  0.479952     0.523397
1.0                         0.526568  0.517093     0.518135
2.0                         0.385287  0.568516     0.549769
3.0                         0.406897  0.462709     0.435582
4.0                         0.288280  0.635616     0.702740
5.0                         0.374315  1.000000     0.576517

In [20]:
#Does seattle's Airbnb has higher price tag than Boston's airbnb
dfBostonAirbnb = dfBostonListing[['bedrooms', 'price']]
dfSeattleAirbnb = dfSeattleListing[['bedrooms', 'price']]

In [21]:
#assign location to each dataframe
dfBostonAirbnb.insert(loc=1, column='Location', value='Boston')
dfSeattleAirbnb.insert(loc=1, column='Location', value='Seattle')

In [27]:
#Concate the two dataframe together
dfAreaAnalysis = dfSeattleAirbnb
dfAreaAnalysis = dfAreaAnalysis.append(dfBostonAirbnb, ignore_index=True)

In [28]:
dfAreaAnalysis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7403 entries, 0 to 7402
Data columns (total 3 columns):
bedrooms    7387 non-null float64
Location    7403 non-null object
price       7403 non-null object
dtypes: float64(1), object(2)
memory usage: 173.6+ KB


In [31]:
dfAreaAnalysis['price'] = dfAreaAnalysis['price'].astype(str)
dfAreaAnalysis['price'] = dfAreaAnalysis['price'].apply(lambda x: x.replace('$','')).apply(lambda x: x.replace(',','')).astype(float)

In [33]:
dfAreaAnalysis.groupby(['bedrooms', 'Location']).mean().unstack()

price            
Location      Boston     Seattle
bedrooms                        
0.0       154.668990  103.548387
1.0       136.285834   95.706247
2.0       258.753247  174.062500
3.0       316.419355  249.530035
4.0       389.400000  313.695652
5.0       429.000000  441.000000
6.0              NaN  578.166667
7.0              NaN  375.000000